## Object Abstraction
A central concept in object abstraction is a *generic function*: a function that can accept values of multiple different types.    

Three techniques for implementing generic function:
- shared interfaces   
- type dispatching 
- type coercion 

### String Representations
By the metaphor of OOP, an object value should behave like the kind of data it is meant to represent.    
It should produce a string representation of itself.    
In Python, all objects produce 2 string representations:    
- `str` is human-readable
- `repr` is Python interpreter-readable   
They are often the same, not always

`repr(object)->a Python expresssion(a string)`
- can be called on **any object** producing strings
- return the canonical string representation of the object, what Python prints in an interactive session
- For most object types, `eval(repr(object)) == object`


In [28]:
12e12

12000000000000.0

In [42]:
repr(12e12)

'12000000000000.0'

In [44]:
s = 's'
s

's'

In [45]:
repr(s)

"'s'"

In [46]:
eval(repr(s))

's'

In [48]:
repr(repr(s))

'"\'s\'"'

In [49]:
eval(eval(repr(repr(s))))

's'

- Some objects do not have a simple Python readable string, like compound objects(class, function), as they can not be written into a simple expression.

`<>` indicate it's not a Python-readble,but a human-readable description

In [34]:
repr(min)

'<built-in function min>'

In [35]:
from fractions import Fraction
half = Fraction(1, 2)
repr(half)

'Fraction(1, 2)'

In [39]:
half

Fraction(1, 2)

In [51]:
eval(repr(half))

Fraction(1, 2)

`str(object) -> string(human-readable)`
- return what Python prints using `print`

In [36]:
str(half)

'1/2'

In [38]:
print(half)

1/2


In [41]:
eval(str(half))

0.5

`eval('python expression')`

### F-Strings
String interpolation involves evaluating a string literal that contains exprssions.   
Using string concatenation:

In [54]:
from math import pi
'pi starts with ' + str(pi) + '...'

'pi starts with 3.141592653589793...'

Using string interpolation:

In [56]:
f'pi starts with {pi}...'

'pi starts with 3.141592653589793...'

### Polymorphic Functions
So when we want to define the `repr` function, we hope we can apply all data types.    
*Polymorphic function*: A function that can be applied on many(poly) different forms(morph) of data.    
e.g. `repr`, `str`: apply to any object   
how?     
The object system provide a solution:   
`repr` invokes `__repr__` method on its argument   
`str` invokes `__str__` method on its argument

In [59]:
half.__repr__()

'Fraction(1, 2)'

In [60]:
repr(half)

'Fraction(1, 2)'

`repr`:
- An instance attribute called __repr__ is ignored!(As we dont want the users of the class to use underscore method). Only class attributes are found.

In [66]:
def repr(x):
    return type(x).__repr__(x)#since self is not bound

In [65]:
type(half).__repr__(half)

'Fraction(1, 2)'

`str`:
- An instance attribute called `__str__` is ignored
- If no `__str__` attribute is found, uses `repr` string

In [71]:
class Bear:
    def __init__(self):
        self.__repr__ = lambda: 'oski'
    def __repr__(self):
        return 'Bear()'
    def __str__(self):
        return 'a bear'

In [72]:
oski = Bear()
print(oski)
print(str(oski))
print(repr(oski))
print(oski.__str__())
print(oski.__repr__())
#可观察到print(object)的interface是class里的__str__

a bear
a bear
Bear()
a bear
oski


### Interface
**Message passing**: Objects interact by looking up attributes on each other   
A shared attribute name helps abstraction.    
An interface is a set of shared attributes.    
e.g. `__repr__` and `__str__` give an interface for producing string representations.

### Special Method Names
(particular to Python)   
Names that start and end with two underscores have built-in behavior.   
`__init__` : method invoked automatically when an object is constructed  
`__repr__`: method invoked to display the Python expression
`__add__`, `__bool__`, `__float__`

In [73]:
zero, one, two = 0, 1, 2
one + two

3

In [75]:
one.__add__(two)

3

In [74]:
bool(zero), bool(one)

(False, True)

In [76]:
zero.__bool__(), one.__bool__()

(False, True)

The special names give us interfaces to be overidden.

In [110]:
from math import gcd
class Ratio:
    def __init__(self, n, d):
        self.numer = n
        self.denom = d
    def __repr__(self):
        return 'Ratio({0}/{1})'.format(self.numer, self.denom)
    def __str__(self):
        return '{0}/{1}'.format(self.numer, self.denom)
    def __add__(self, other):
        #type dispatching
        if isinstance(other, int):
            n = self.numer + self.denom * other
            d = self.denom
        elif isinstance(other, Ratio):
            n = self.numer * other.denom + self.denom * other.numer
            d = self.denom * other.denom
        elif isinstance(other, float):
            return float(self) + other
            #type coercion
        g = gcd(n, d)
        return Ratio(n//g, d//g)
    __radd__ = __add__
    def __float__(self):
        return self.numer/self.denom



In [111]:
half = Ratio(1, 2)
quater = Ratio(1, 4)

In [112]:
half + quater

Ratio(3/4)

In [113]:
2 + half

Ratio(5/2)

In [114]:
2.5 + half

3.0